In [20]:
# REDFIELD.py: Time-Dependent Redfield Solver for D Wave Simulations

from qutip import *
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Parameters specific to the D Wave 2X onsite.
dWaveAnnealingSchedule = pd.read_csv('~/Documents/LANLA/DWaveAnnealingSchedule.csv', sep=',',header=None)
annealingParameter = dWaveAnnealingSchedule[0]
A = [(10**9)*x for x in dWaveAnnealingSchedule[1]]
B = [(10**9)*x for x in dWaveAnnealingSchedule[2]]

# Fundamental Constants
kB = Quantity[1.38065 10^-23, ("Joules")/("Kelvins")];
T = Quantity[15.5 10^-3, "Kelvin"];
hbar = Quantity[6.62607 10^-34, "Joules" "Seconds"];
\[Beta] = 1/(k T);

In [32]:
#Define Pauli matrices. e.g. X(actingQubit) is the bit flip operator on actingQubit.

def X(actingQubit, numQubits):
    qubits = range(numQubits)
    if actingQubit >= numQubits:
        print "Error. Pauli matrix over-indexed"
    else:
        def XTensor(actingQubit, qubit):
            if qubit == actingQubit:
                return sigmax()
            else: 
                return identity(2)
        return tensor([XTensor(actingQubit, qubit) for qubit in qubits])

def Y(actingQubit, numQubits):
    qubits = range(numQubits)
    if actingQubit >= numQubits:
        print "Error. Pauli matrix over-indexed"
    else:
        def YTensor(actingQubit, qubit):
            if qubit == actingQubit:
                return sigmay()
            else: 
                return identity(2)
        return tensor([YTensor(actingQubit, qubit) for qubit in qubits])

def Z(actingQubit, numQubits):
    qubits = range(numQubits)
    if actingQubit >= numQubits:
        print "Error. Pauli matrix over-indexed"
    else:
        def ZTensor(actingQubit, qubit):
            if qubit == actingQubit:
                return sigmaz()
            else: 
                return identity(2)
        return tensor([ZTensor(actingQubit, qubit) for qubit in qubits])

In [54]:
#Define D Wave Hamiltonian
def bulkCoupling(J, K, qubitIndex, numQubits):
    if qubitIndex >= numQubits - 1:
        print "Error. coupler is over-indexed"
    else:
        if numQubits % 2 == 0:
            if qubitIndex in [numQubits/2 - 1, numQubits/2]:
                return -J
            else:
                return -K
        else:
            if qubitIndex in [numQubits/2 - 1, numQubits/2]:
                return -J
            else:
                return -K
               
def dWaveHamiltonian(s, I, J, K, numQubits):
    qubits = range(numQubits)
    sRescaled = int(max(0, s*round(len(annealingParameter)) - 1))
    bulkTerms = sum([bulkCoupling(J, K, qubit, numQubits)*Z(qubit, numQubits)*Z(qubit + 1, numQubits) for qubit in range(numQubits - 1)])
    boundaryTerm = I*Z(qubits[-1], numQubits)*Z(qubits[0], numQubits)
    rawProblemHamiltonian = bulkTerms + boundaryTerm
    rawDriverHamiltonian = sum([X(qubit, numQubits) for qubit in qubits])
    problemHamiltonian = 2*np.pi*(B[sRescaled]/2)*rawProblemHamiltonian
    driverHamiltonian = 2*np.pi*(A[sRescaled]/2)*rawDriverHamiltonian
    return problemHamiltonian + driverHamiltonian
                    
# 

In [81]:
# Redfield Timing
import time

start = time.time()
H = dWaveHamiltonian(0.5,.2,.3, 1, 15)
end = time.time()

print end-start
H

0.526549816132


Quantum object: dims = [[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]], shape = (32768, 32768), type = oper, isherm = True
Qobj data =
  (0, 0)	(-126606183940+0j)
  (0, 1)	(4241150082.35+0j)
  (0, 2)	(4241150082.35+0j)
  (0, 4)	(4241150082.35+0j)
  (0, 8)	(4241150082.35+0j)
  (0, 16)	(4241150082.35+0j)
  (0, 32)	(4241150082.35+0j)
  (0, 64)	(4241150082.35+0j)
  (0, 128)	(4241150082.35+0j)
  (0, 256)	(4241150082.35+0j)
  (0, 512)	(4241150082.35+0j)
  (0, 1024)	(4241150082.35+0j)
  (0, 2048)	(4241150082.35+0j)
  (0, 4096)	(4241150082.35+0j)
  (0, 8192)	(4241150082.35+0j)
  (0, 16384)	(4241150082.35+0j)
  (1, 0)	(4241150082.35+0j)
  (1, 1)	(-110269902141+0j)
  (1, 3)	(4241150082.35+0j)
  (1, 5)	(4241150082.35+0j)
  (1, 9)	(4241150082.35+0j)
  (1, 17)	(4241150082.35+0j)
  (1, 33)	(4241150082.35+0j)
  (1, 65)	(4241150082.35+0j)
  (1, 129)	(4241150082.35+0j)
  :	:
  (32766, 32638)	(4241150082.35+0j)
  (32766, 32702)	(4241150082.35+0j)
  (32766, 